In [ ]:
#!pip install h2o

In [1]:
import os
PATH = os.getcwd()

In [2]:
os.chdir(PATH)

In [3]:
import h2o

In [4]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
; Java HotSpot(TM) Client VM (build 25.211-b12, mixed mode)


C:\Users\ANEEL\AppData\Local\Continuum\anaconda3\lib\site-packages\h2o\backend\server.py:369: UserWarning:   You have a 32-bit version of Java. H2O works best with 64-bit Java.
  Please download the latest 64-bit Java SE JDK from Oracle.

  warn("  You have a 32-bit version of Java. H2O works best with 64-bit Java.\n"


  Starting server from C:\Users\ANEEL\AppData\Local\Continuum\anaconda3\lib\site-packages\h2o\backend\bin\h2o.jar
  Ice root: C:\Users\ANEEL\AppData\Local\Temp\tmpm_6dvw1p
  JVM stdout: C:\Users\ANEEL\AppData\Local\Temp\tmpm_6dvw1p\h2o_ANEEL_started_from_python.out
  JVM stderr: C:\Users\ANEEL\AppData\Local\Temp\tmpm_6dvw1p\h2o_ANEEL_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O cluster uptime:,10 secs
H2O cluster timezone:,Asia/Kolkata
H2O data parsing timezone:,UTC
H2O cluster version:,3.24.0.3
H2O cluster version age:,"14 days, 4 hours and 15 minutes"
H2O cluster name:,H2O_from_python_ANEEL_0y6u5s
H2O cluster total nodes:,1
H2O cluster free memory:,247.5 Mb
H2O cluster total cores:,4
H2O cluster allowed cores:,4
H2O cluster status:,"accepting new members, healthy"


import pandas as pd
df= pd.read_csv('Train.csv', delimiter = ',')

In [7]:
loan_csv = "C://Users//ANEEL//Desktop//PhDhackethon0519//PHD_TrainData_and_ProblemStatement-1558070454703//PHD_TrainData_and_ProblemStatement//Train.csv" 

loan_csv = "C://Users//ANEEL//Desktop//ANN AI//H2O//Train.csv" 

In [8]:
data = h2o.import_file(loan_csv)

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [9]:
data.shape

(42582, 6)

In [13]:
data.describe()

Rows:42582
Cols:6




,ReportID,SalesPersonID,ProductID,Quantity,TotalSalesValue,Suspicious
type,string,enum,enum,int,int,enum
mins,NaN,,,33.0,598.0,
mean,NaN,,,4910.0483537645,16209.229885867271,
maxs,NaN,,,19708130.0,3953985.0,
sigma,NaN,,,98336.21452586878,59971.95129694922,
zeros,0,,,0,0,
missing,0,0,0,0,0,0
0,Rep10101,C21116,PR6112,182.0,1665.0,indeterminate
1,Rep10102,C21116,PR6112,182.0,1740.0,indeterminate
2,Rep10103,C21116,PR6253,101.0,1040.0,indeterminate


In [15]:
data.columns

['ReportID',
 'SalesPersonID',
 'ProductID',
 'Quantity',
 'TotalSalesValue',
 'Suspicious']

In [16]:
data['Suspicious'] = data['Suspicious'].asfactor()  #encode the binary repsonse as a factor
data['Suspicious'].levels()

[['No', 'Yes', 'indeterminate']]

In [17]:

# Partition data into 70%, 15%, 15% chunks
# Setting a seed will guarantee reproducibility

splits = data.split_frame(ratios=[0.7, 0.15], seed=1)  

train = splits[0]
valid = splits[1]
test = splits[2]

In [18]:
print(train.nrow, valid.nrow, test.nrow)

29913 6282 6387


In [19]:
y = 'Suspicious'
x = list(data.columns)

In [20]:
x

['ReportID',
 'SalesPersonID',
 'ProductID',
 'Quantity',
 'TotalSalesValue',
 'Suspicious']

In [21]:
x.remove(y)  #remove the response
x.remove('ReportID')  #remove the interest rate column because it's correlated with the outcome
#x.remove('IV')

In [22]:
x

['SalesPersonID', 'ProductID', 'Quantity', 'TotalSalesValue']

# H2O Machine Learning¶
Now that we have prepared the data, we can train some models. We will start by training a single model from each of the H2O supervised algos:

Generalized Linear Model (GLM)
Random Forest (RF)
Gradient Boosting Machine (RF)
Deep Learning (DL)
Naive Bayes (NB)
1. Generalized Linear Model
Let's start with a basic binomial Generalized Linear Model (GLM). By default, H2O's GLM uses a regularized, elastic net model.

# Train a default GLM
We first create an object of class, "H2OGeneralizedLinearEstimator". This does not actually do any training, it just sets the model up for training by specifying model parameters.

# 2. Random Forest¶
H2O's Random Forest (RF) is implements a distributed version of the standard Random Forest algorithm and variable importance measures.

In [26]:
# Import H2O RF:
from h2o.estimators.random_forest import H2ORandomForestEstimator

In [27]:
# Initialize the RF estimator:

rf_fit1 = H2ORandomForestEstimator(model_id='rf_fit1', seed=1)

In [28]:
rf_fit1.train(x=x, y=y, training_frame=train)

drf Model Build progress: |███████████████████████████████████████████████| 100%


In [29]:
rf_fit2 = H2ORandomForestEstimator(model_id='rf_fit2', ntrees=100, seed=1)
rf_fit2.train(x=x, y=y, training_frame=train)

drf Model Build progress: |███████████████████████████████████████████████| 100%


# Compare model performance¶
Let's compare the performance of the two RFs that were just trained.

In [30]:
rf_perf1 = rf_fit1.model_performance(test)
rf_perf2 = rf_fit2.model_performance(test)

In [32]:
print(rf_perf1)


ModelMetricsMultinomial: drf
** Reported on test data. **

MSE: 0.038427376770031677
RMSE: 0.19602902022412824
LogLoss: 0.3356651376044006
Mean Per-Class Error: 0.48848650339318755
Confusion Matrix: Row labels: Actual class; Column labels: Predicted class



No,Yes,indeterminate,Error,Rate
207.0,0.0,196.0,0.4863524,196 / 403
1.0,1.0,31.0,0.9696970,32 / 33
55.0,1.0,5895.0,0.0094102,"56 / 5,951"
263.0,2.0,6122.0,0.0444653,"284 / 6,387"


Top-3 Hit Ratios: 


k,hit_ratio
1,0.9555347
2,0.9923282
3,1.0


In [33]:
# Retreive test set AUC
print(rf_perf2)
#print(rf_perf2.auc())


ModelMetricsMultinomial: drf
** Reported on test data. **

MSE: 0.03817727884166911
RMSE: 0.1953900684315073
LogLoss: 0.3049651779170323
Mean Per-Class Error: 0.4759041037140137
Confusion Matrix: Row labels: Actual class; Column labels: Predicted class



No,Yes,indeterminate,Error,Rate
210.0,0.0,193.0,0.4789082,193 / 403
1.0,2.0,30.0,0.9393939,31 / 33
55.0,1.0,5895.0,0.0094102,"56 / 5,951"
266.0,3.0,6118.0,0.0438390,"280 / 6,387"


Top-3 Hit Ratios: 


k,hit_ratio
1,0.9561610
2,0.9910756
3,1.0


# Cross-validate performance
Rather than using held-out test set to evaluate model performance, a user may wish to estimate model performance using cross-validation. Using the RF algorithm (with default model parameters) as an example, we demonstrate how to perform k-fold cross-validation using H2O. No custom code or loops are required, you simply specify the number of desired folds in the nfolds argument.

Since we are not going to use a test set here, we can use the original (full) dataset, which we called data rather than the subsampled  train dataset. Note that this will take approximately k (nfolds) times longer than training a single RF model, since it will train k models in the cross-validation process (trained on n(k-1)/k rows), in addition to the final model trained on the full training_frame dataset with n rows.

In [34]:
rf_fit3 = H2ORandomForestEstimator(model_id='rf_fit3', seed=1, nfolds=5)
rf_fit3.train(x=x, y=y, training_frame=data)

drf Model Build progress: |███████████████████████████████████████████████| 100%


In [35]:
print(rf_fit3)

Model Details
H2ORandomForestEstimator :  Distributed Random Forest
Model Key:  rf_fit3


ModelMetricsMultinomial: drf
** Reported on train data. **

MSE: 0.03322321720995297
RMSE: 0.18227237094511325
LogLoss: 0.33399120802298166
Mean Per-Class Error: 0.47542582594081195
Confusion Matrix: Row labels: Actual class; Column labels: Predicted class



No,Yes,indeterminate,Error,Rate
1339.0,1.0,1215.0,0.4759295,"1,216 / 2,555"
9.0,10.0,162.0,0.9447514,171 / 181
217.0,6.0,39623.0,0.0055965,"223 / 39,846"
1565.0,17.0,41000.0,0.0378094,"1,610 / 42,582"


Top-3 Hit Ratios: 


k,hit_ratio
1,0.9621906
2,0.991851
3,0.9999999



ModelMetricsMultinomial: drf
** Reported on cross-validation data. **

MSE: 0.03548069400027559
RMSE: 0.18836319704304127
LogLoss: 0.23863728866391917
Mean Per-Class Error: 0.4935289250231187
Confusion Matrix: Row labels: Actual class; Column labels: Predicted class



...

,mean,sd,cv_1_valid,cv_2_valid,cv_3_valid,cv_4_valid,cv_5_valid
accuracy,0.9607711,0.0014159,0.9600890,0.9626433,0.9571528,0.9618465,0.9621239
err,0.0392289,0.0014159,0.0399111,0.0373567,0.0428472,0.0381536,0.0378761
err_count,334.2,13.663089,341.0,316.0,369.0,324.0,321.0
logloss,0.2385282,0.0227098,0.2055965,0.2381961,0.2862584,0.2603634,0.2022268
max_per_class_error,0.9779416,0.0079189,0.969697,0.972973,1.0,0.9756098,0.9714286
mean_per_class_accuracy,0.5066611,0.0070334,0.5066772,0.5130088,0.4873966,0.5129521,0.5132709
mean_per_class_error,0.4933389,0.0070334,0.4933228,0.4869911,0.5126034,0.4870479,0.4867291
mse,0.0354727,0.0009298,0.0351674,0.0343698,0.0380081,0.0352784,0.0345398
r2,0.8449727,0.0024190,0.8494428,0.8463606,0.8392865,0.8434275,0.8463460
rmse,0.1883103,0.0024413,0.1875298,0.1853908,0.1949568,0.1878253,0.1858489


Scoring History: 


,timestamp,duration,number_of_trees,training_rmse,training_logloss,training_classification_error
,2019-05-21 07:57:28,2 min 15.964 sec,0.0,nan,nan,nan
,2019-05-21 07:57:29,2 min 16.351 sec,1.0,0.2378541,1.6972444,0.0504805
,2019-05-21 07:57:29,2 min 16.700 sec,2.0,0.2300361,1.5600611,0.0485581
,2019-05-21 07:57:29,2 min 17.079 sec,3.0,0.2237315,1.3976178,0.0475292
,2019-05-21 07:57:30,2 min 17.468 sec,4.0,0.2190606,1.2728291,0.0474646
,2019-05-21 07:57:30,2 min 17.849 sec,5.0,0.2141252,1.1525410,0.0466020
,2019-05-21 07:57:30,2 min 18.217 sec,6.0,0.2094935,1.0442167,0.0459070
,2019-05-21 07:57:31,2 min 18.546 sec,7.0,0.2064106,0.9677788,0.0452934
,2019-05-21 07:57:31,2 min 18.900 sec,8.0,0.2025799,0.9013860,0.0445001
,2019-05-21 07:57:31,2 min 19.232 sec,9.0,0.1994482,0.8255993,0.0438423


Variable Importances: 


variable,relative_importance,scaled_importance,percentage
SalesPersonID,54424.9843750,1.0,0.3729654
ProductID,48713.1210938,0.8950507,0.3338230
TotalSalesValue,33442.6718750,0.6144728,0.2291771
Quantity,9344.2363281,0.1716902,0.0640345


# 3. Gradient Boosting Machine¶
H2O's Gradient Boosting Machine (GBM) offers a Stochastic GBM, which can increase performance quite a bit compared to the original GBM implementation.

In [36]:
# Import H2O GBM:
from h2o.estimators.gbm import H2OGradientBoostingEstimator

In [37]:

# Initialize and train the GBM estimator:

gbm_fit1 = H2OGradientBoostingEstimator(model_id='gbm_fit1', seed=1)
gbm_fit1.train(x=x, y=y, training_frame=train)

gbm Model Build progress: |███████████████████████████████████████████████| 100%


# Train a GBM with more trees
Next we will increase the number of trees used in the GBM by setting ntrees=500. The default number of trees in an H2O GBM is 50, so this GBM will trained using ten times the default. Increasing the number of trees in a GBM is one way to increase performance of the model, however, you have to be careful not to overfit your model to the training data by using too many trees. To automatically find the optimal number of trees, you must use H2O's early stopping functionality. This example will not do that, however, the following example will.

In [38]:
gbm_fit2 = H2OGradientBoostingEstimator(model_id='gbm_fit2', ntrees=500, seed=1)
gbm_fit2.train(x=x, y=y, training_frame=train)

gbm Model Build progress: |███████████████████████████████████████████████| 100%


# Train a GBM with early stopping
We will again set ntrees = 500, however, this time we will use early stopping in order to prevent overfitting (from too many trees). All of H2O's algorithms have early stopping available, however, with the exception of Deep Learning, it is not enabled by default.

There are several parameters that should be used to control early stopping. The three that are generic to all the algorithms are: stopping_rounds, stopping_metric and stopping_tolerance. The stopping metric is the metric by which you'd like to measure performance, and so we will choose AUC here. The score_tree_interval is a parameter specific to Random Forest and GBM. Setting score_tree_interval=5 will score the model after every five trees. The parameters we have set below specify that the model will stop training after there have been three scoring intervals where the AUC has not increased more than 0.0005. Since we have specified a validation frame, the stopping tolerance will be computed on validation AUC rather than training AUC.

# Compare model performance
Let's compare the performance of the three GBMs that were just trained.

In [39]:
gbm_perf1 = gbm_fit1.model_performance(test)
gbm_perf2 = gbm_fit2.model_performance(test)
#gbm_perf3 = gbm_fit3.model_performance(test)

In [40]:
print(gbm_perf1)


ModelMetricsMultinomial: gbm
** Reported on test data. **

MSE: 0.041510998143212956
RMSE: 0.20374247996726888
LogLoss: 0.17701441684620092
Mean Per-Class Error: 0.5008169604198464
Confusion Matrix: Row labels: Actual class; Column labels: Predicted class



No,Yes,indeterminate,Error,Rate
179.0,1.0,223.0,0.5558313,224 / 403
2.0,2.0,29.0,0.9393939,31 / 33
42.0,1.0,5908.0,0.0072257,"43 / 5,951"
223.0,4.0,6160.0,0.0466573,"298 / 6,387"


Top-3 Hit Ratios: 


k,hit_ratio
1,0.9533427
2,0.9951464
3,1.0


In [41]:
print(gbm_perf2)


ModelMetricsMultinomial: gbm
** Reported on test data. **

MSE: 0.0402397613218532
RMSE: 0.20059850777573895
LogLoss: 0.21129761092657223
Mean Per-Class Error: 0.46798194815605826
Confusion Matrix: Row labels: Actual class; Column labels: Predicted class



No,Yes,indeterminate,Error,Rate
221.0,0.0,182.0,0.4516129,182 / 403
2.0,2.0,29.0,0.9393939,31 / 33
71.0,6.0,5874.0,0.0129390,"77 / 5,951"
294.0,8.0,6085.0,0.0454047,"290 / 6,387"


Top-3 Hit Ratios: 


k,hit_ratio
1,0.9545953
2,0.9954595
3,0.9999999


# Scoring History
To examine the scoring history, use the scoring_history method on a trained model. If score_tree_interval is not specified, it will score at various intervals, as we can see for gbm_fit2.scoring_history() below. However, regular 5-tree intervals are used for gbm_fit3.scoring_history().

The gbm_fit2 was trained only using a training set (no validation set), so the scoring history is calculated for training set performance metrics only.

In [42]:
gbm_fit2.scoring_history()

,,timestamp,duration,number_of_trees,training_rmse,training_logloss,training_classification_error
0,,2019-05-21 08:06:34,0.001 sec,0.0,0.666667,1.098612,0.119647
1,,2019-05-21 08:06:34,0.133 sec,1.0,0.605909,0.931734,0.049076
2,,2019-05-21 08:06:35,0.277 sec,2.0,0.550646,0.801284,0.047872
3,,2019-05-21 08:06:35,0.381 sec,3.0,0.500933,0.696792,0.049042
4,,2019-05-21 08:06:35,0.497 sec,4.0,0.456413,0.611179,0.047972
5,,2019-05-21 08:06:35,0.591 sec,5.0,0.416992,0.540352,0.046334
6,,2019-05-21 08:06:35,0.708 sec,6.0,0.380980,0.478844,0.045131
7,,2019-05-21 08:06:35,0.815 sec,7.0,0.350741,0.429025,0.044596
8,,2019-05-21 08:06:35,0.916 sec,8.0,0.324189,0.386337,0.044262
9,,2019-05-21 08:06:35,1.049 sec,9.0,0.300620,0.348911,0.043861


When early stopping is used, we see that training stopped at 105 trees instead of the full 500. Since we used a validation set in gbm_fit3, both training and validation performance metrics are stored in the scoring history object. Take a look at the validation AUC to observe that the correct stopping tolerance was enforced.

In [26]:
#gbm_fit3.scoring_history()

# 4. Deep Learning
H2O's Deep Learning algorithm is a multilayer feed-forward artificial neural network. It can also be used to train an autoencoder, however, in the example below we will train a standard supervised prediction model.

In [43]:
# Import H2O DL:
from h2o.estimators.deeplearning import H2ODeepLearningEstimator

# Train a default DL
First we will train a basic DL model with default parameters. DL will infer the response distribution from the response encoding if not specified explicitly through the distribution argument. H2O's DL will not be reproducbible if run on more than a single core, so in this example, the performance metrics below may vary slightly from what you see on your machine.

In H2O's DL, early stopping is enabled by default, so below, it will use the training set and default stopping parameters to perform early stopping.

In [44]:
# Initialize and train the DL estimator:

dl_fit1 = H2ODeepLearningEstimator(model_id='dl_fit1', seed=1)
dl_fit1.train(x=x, y=y, training_frame=train)

deeplearning Model Build progress: |██████████████████████████████████████| 100%


# Train a DL with new architecture and more epochs
Next we will increase the number of epochs used in the GBM by setting epochs=20 (the default is 10). Increasing the number of epochs in a deep neural net may increase performance of the model, however, you have to be careful not to overfit your model. To automatically find the optimal number of epochs, you must use H2O's early stopping functionality. Unlike the rest of the H2O algorithms, H2O's DL will use early by default, so we will first turn it off in the next example by setting stopping_rounds=0, for comparison.

In [45]:
dl_fit2 = H2ODeepLearningEstimator(model_id='dl_fit2', 
                                   epochs=20, 
                                   hidden=[10,10], 
                                   stopping_rounds=0,  #disable early stopping
                                   seed=1)
dl_fit2.train(x=x, y=y, training_frame=train)

deeplearning Model Build progress: |██████████████████████████████████████| 100%


# Train a DL with early stopping¶
This example will use the same model parameters as dl_fit2, however, we will turn on early stopping and specify the stopping criterion. We will also pass a validation set, as is recommended for early stopping.

# Compare model performance¶
Again, we will compare the model performance of the three models using a test set and AUC.

In [47]:
dl_perf1 = dl_fit1.model_performance(test)
dl_perf2 = dl_fit2.model_performance(test)
#dl_perf3 = dl_fit3.model_performance(test)

In [48]:
print(dl_perf1)



ModelMetricsMultinomial: deeplearning
** Reported on test data. **

MSE: 0.05038805672882979
RMSE: 0.22447284185136915
LogLoss: 0.21235794400224686
Mean Per-Class Error: 0.48119545537707836
Confusion Matrix: Row labels: Actual class; Column labels: Predicted class



No,Yes,indeterminate,Error,Rate
212.0,0.0,191.0,0.4739454,191 / 403
3.0,2.0,28.0,0.9393939,31 / 33
177.0,3.0,5771.0,0.0302470,"180 / 5,951"
392.0,5.0,5990.0,0.0629403,"402 / 6,387"


Top-3 Hit Ratios: 


k,hit_ratio
1,0.9370596
2,0.9959292
3,1.0


In [49]:
print(dl_perf2)


ModelMetricsMultinomial: deeplearning
** Reported on test data. **

MSE: 0.0459964692240572
RMSE: 0.21446787457345962
LogLoss: 0.21333269003558103
Mean Per-Class Error: 0.541051201298056
Confusion Matrix: Row labels: Actual class; Column labels: Predicted class



No,Yes,indeterminate,Error,Rate
156.0,0.0,247.0,0.6129032,247 / 403
2.0,0.0,31.0,1.0,33 / 33
61.0,0.0,5890.0,0.0102504,"61 / 5,951"
219.0,0.0,6168.0,0.0533897,"341 / 6,387"


Top-3 Hit Ratios: 


k,hit_ratio
1,0.9466103
2,0.9948333
3,1.0


In [50]:
dl_fit2.scoring_history()

,,timestamp,duration,training_speed,epochs,iterations,samples,training_rmse,training_logloss,training_r2,training_classification_error
0,,2019-05-21 08:24:13,0.000 sec,None,0.000000,0,0.0,NaN,NaN,NaN,NaN
1,,2019-05-21 08:24:14,0.956 sec,13518 obs/sec,0.185739,1,5556.0,0.242618,0.247576,0.737769,0.064638
2,,2019-05-21 08:24:19,6.210 sec,12844 obs/sec,2.239729,12,66997.0,0.222043,0.193754,0.780359,0.057379
3,,2019-05-21 08:24:25,11.622 sec,12664 obs/sec,4.307759,23,128858.0,0.218976,0.183932,0.786385,0.057080
4,,2019-05-21 08:24:30,16.589 sec,12933 obs/sec,6.367533,34,190472.0,0.212389,0.166397,0.799044,0.054296
5,,2019-05-21 08:24:35,21.778 sec,13238 obs/sec,8.622271,46,257918.0,0.214320,0.174542,0.795373,0.055290
6,,2019-05-21 08:24:40,26.938 sec,13398 obs/sec,10.868820,58,325119.0,0.208009,0.167006,0.807246,0.052605
7,,2019-05-21 08:24:45,32.023 sec,13552 obs/sec,13.117106,70,392372.0,0.196392,0.137468,0.828176,0.046738
8,,2019-05-21 08:24:50,37.230 sec,13650 obs/sec,15.371344,82,459803.0,0.210860,0.171546,0.801927,0.052506
9,,2019-05-21 08:24:55,42.398 sec,13731 obs/sec,17.622271,94,527135.0,0.193296,0.138870,0.833550,0.045545


# 4. Naive Bayes¶
The Naive Bayes (NB) algorithm does not usually beat an algorithm like a Random Forest or GBM, however it is still a popular algorithm, especially in the text domain (when your input is text encoded as "Bag of Words", for example). The Naive Bayes algorithm is for binary or multiclass classification problems only, not regression. Therefore, your response must be a factor instead of numeric.

In [51]:
# Import H2O NB:
from h2o.estimators.naive_bayes import H2ONaiveBayesEstimator

# Train a default NB¶
First we will train a basic NB model with default parameters.

In [52]:
# Initialize and train the NB estimator:

nb_fit1 = H2ONaiveBayesEstimator(model_id='nb_fit1')
nb_fit1.train(x=x, y=y, training_frame=train)

naivebayes Model Build progress: |████████████████████████████████████████| 100%


# Train a NB model with Laplace Smoothing¶
One of the few tunable model parameters for the Naive Bayes algorithm is the amount of Laplace smoothing. The H2O Naive Bayes model will not use any Laplace smoothing by default.

In [53]:
nb_fit2 = H2ONaiveBayesEstimator(model_id='nb_fit2', laplace=6)
nb_fit2.train(x=x, y=y, training_frame=train)

naivebayes Model Build progress: |████████████████████████████████████████| 100%


# Compare model performance¶
We will compare the model performance of the two NB models using test set AUC.

In [54]:
nb_perf1 = nb_fit1.model_performance(test)
nb_perf2 = nb_fit2.model_performance(test)

In [55]:
print(nb_perf1)



ModelMetricsMultinomial: naivebayes
** Reported on test data. **

MSE: 0.07533331359241434
RMSE: 0.2744691487078545
LogLoss: 0.521746259423089
Mean Per-Class Error: 0.6144492585968945
Confusion Matrix: Row labels: Actual class; Column labels: Predicted class



No,Yes,indeterminate,Error,Rate
49.0,12.0,342.0,0.8784119,354 / 403
8.0,2.0,23.0,0.9393939,31 / 33
121.0,31.0,5799.0,0.0255419,"152 / 5,951"
178.0,45.0,6164.0,0.0840770,"537 / 6,387"


Top-3 Hit Ratios: 


k,hit_ratio
1,0.9159229
2,0.9812118
3,0.9999999


In [56]:
print(nb_perf2)


ModelMetricsMultinomial: naivebayes
** Reported on test data. **

MSE: 0.07608316612820885
RMSE: 0.27583177142636933
LogLoss: 0.5589284219951537
Mean Per-Class Error: 0.6085697936511841
Confusion Matrix: Row labels: Actual class; Column labels: Predicted class



No,Yes,indeterminate,Error,Rate
42.0,10.0,351.0,0.8957816,361 / 403
5.0,3.0,25.0,0.9090909,30 / 33
97.0,27.0,5827.0,0.0208368,"124 / 5,951"
144.0,40.0,6203.0,0.0806325,"515 / 6,387"


Top-3 Hit Ratios: 


k,hit_ratio
1,0.9193675
2,0.9835604
3,1.0


In [57]:
nb_fit1.predict(data)

naivebayes prediction progress: |█████████████████████████████████████████| 100%


predict,No,Yes,indeterminate
indeterminate,0.00725286,0.00022584,0.992521
indeterminate,0.0072516,0.000225758,0.992523
indeterminate,0.0552508,0.000362816,0.944386
indeterminate,0.0548875,0.000357167,0.944755
indeterminate,0.00752753,0.000234556,0.992238
indeterminate,0.0548094,0.00035474,0.944836
indeterminate,0.00752032,0.000234089,0.992246
indeterminate,0.0552394,0.000362664,0.944398
indeterminate,0.0072516,0.000225758,0.992523
indeterminate,0.00458105,0.000142426,0.995277


In [61]:
nb_fit2.predict(data)

naivebayes prediction progress: |█████████████████████████████████████████| 100%


predict,No,Yes,indeterminate
indeterminate,0.00776785,0.000224908,0.992007
indeterminate,0.0077665,0.000224825,0.992009
indeterminate,0.0197265,2.38413e-05,0.98025
indeterminate,0.0195918,2.34641e-05,0.980385
indeterminate,0.00804658,0.00023314,0.99172
indeterminate,0.0195629,2.33034e-05,0.980414
indeterminate,0.00803887,0.000232676,0.991728
indeterminate,0.0197223,2.38311e-05,0.980254
indeterminate,0.0077665,0.000224825,0.992009
indeterminate,0.00500152,0.000144589,0.994854


In [64]:
nb_test= nb_fit2.predict(test)

naivebayes prediction progress: |█████████████████████████████████████████| 100%


# Run AutoML
Run AutoML, stopping after 10 models. The max_models argument specifies the number of individual (or "base") models, and does not include the two ensemble models that are trained at the end.

# Leaderboard¶
Next, we will view the AutoML Leaderboard. Since we did not specify a leaderboard_frame in the H2OAutoML.train() method for scoring and ranking the models, the AutoML leaderboard uses cross-validation metrics to rank the models.

A default performance metric for each machine learning task (binary classification, multiclass classification, regression) is specified internally and the leaderboard will be sorted by that metric. In the case of binary classification, the default ranking metric is Area Under the ROC Curve (AUC). In the future, the user will be able to specify any of the H2O metrics so that different metrics can be used to generate rankings on the leaderboard.

The leader model is stored at aml.leader and the leaderboard is stored at aml.leaderboard.

Now we will view a snapshot of the top models. Here we should see the two Stacked Ensembles at or near the top of the leaderboard. Stacked Ensembles can almost always outperform a single model.

# Ensemble Exploration¶
To understand how the ensemble works, let's take a peek inside the Stacked Ensemble "All Models" model. The "All Models" ensemble is an ensemble of all of the individual models in the AutoML run. This is often the top performing model on the leaderboard.

Examine the variable importance of the metalearner (combiner) algorithm in the ensemble. This shows us how much each base learner is contributing to the ensemble. The AutoML Stacked Ensembles use the default metalearner algorithm (GLM with non-negative weights), so the variable importance of the metalearner is actually the standardized coefficient magnitudes of the GLM.

# Save Leader Model¶
There are two ways to save the leader model -- binary format and MOJO format. If you're taking your leader model to production, then we'd suggest the MOJO format since it's optimized for production use.

In [36]:
h2o.shutdown(prompt=False)

[WARNING] in <ipython-input-36-02ff3806b010> line 1:
    >>> h2o.shutdown(prompt=False)
        ^^^^ Deprecated, use ``h2o.cluster().shutdown()``.
H2O session _sid_9b45 closed.
